In [138]:
from playwright.async_api import async_playwright
from playwright_stealth import stealth_sync , stealth_async


In [139]:

pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)


In [140]:
import os
os.makedirs('playwright/.auth', exist_ok=True)
if os.path.exists('playwright/.auth/state.json'):
    context = await browser.new_context(storage_state="playwright/.auth/state.json")
    page = await context.new_page()


In [141]:
async def extract_info(page):
    container = await page.query_selector('.e1usz6i03')
    element = await container.query_selector('#phone-number-content')
    phone_number  = await element.inner_text()
    return phone_number

In [142]:
async def search_person(page, firstName, lastName, middleName="", city=None, state=None):
    # https://www.spokeo.com/Samama-Mahmud?middle_name=B
    url = f"https://www.spokeo.com/{firstName}-{lastName}?middle_name={middleName}"
    await page.goto(url)    
    link_elements = await page.query_selector_all('.e1ndw42t0')
    # links = [
    #     link_element.get_attribute('href')
    #     for link_element in link_elements
    # ]
    links = []
    for link_element in link_elements:
        link = await link_element.get_attribute('href')
        links.append(link)

    
    return links

In [143]:
firstName = "Samama"
lastName = "Mahmud"
middleName = "B"
user_page  = await search_person(page,firstName, lastName,middleName)
print(user_page)

['/Samama-Mahmud/Wisconsin/Milwaukee/p2015111502099087352946730813837']


In [144]:
url_user = "https://www.spokeo.com/Samama-Mahmud/Wisconsin/Milwaukee/p2015111502099087352946730813837"
await page.goto(url_user)  



<Response url='https://www.spokeo.com/Samama-Mahmud/Wisconsin/Milwaukee/p2015111502099087352946730813837' request=<Request url='https://www.spokeo.com/Samama-Mahmud/Wisconsin/Milwaukee/p2015111502099087352946730813837' method='GET'>>

In [145]:
# phone_number = await extract_info(page)
# phone_number

container = await page.query_selector('.e1usz6i03')
container

<JSHandle preview=JSHandle@node>

In [146]:
el = await container.query_selector('#phone-number-content')

In [147]:
await el.inner_text()

'(847) 581-0171'

In [172]:
async def preson_search(page, firstName, lastName, middleName="", city=None, state=None):
    # https://www.spokeo.com/Samama-Mahmud?middle_name=B
    url = f"https://www.spokeo.com/{firstName}-{lastName}?middle_name={middleName}"
    await page.goto(url)    
    link_elements = await page.query_selector_all('.e1ndw42t0')
    # links = [
    #     link_element.get_attribute('href')
    #     for link_element in link_elements
    # ]
    links = []
    for link_element in link_elements:
        link = await link_element.get_attribute('href')
        links.append(f"https://www.spokeo.com{link}")
        
    phones = []
    for link in links:
        await page.goto(link)
        try:
            phone_number = await extract_info(page)
            phones.append(phone_number)
        except Exception as e:
            print(e,"==>", link)
            pass
 
    return phones

In [173]:
firstName = "Samama"
lastName = "Mahmud"
middleName = "B"
user_page  = await preson_search(page,firstName, lastName,middleName)
user_page

['(847) 581-0171']

# Leads Analysis


In [174]:
#load data into a dataframe
import pandas as pd
df = pd.read_csv('leads_not_found.csv')
df.shape

(107, 34)

In [175]:
df.head()

,id,case_id,first_name,last_name,middle_name,court_code,age,year_of_birth,email,phone,...,notes,violation,court,accidentCheckbox,commercialDriverLicence,ticket_img,user_id,report,details,cloudtalk_upload
0,NaN,DICKERSON__JOHN_E_63_01_02_2024,JOHN,DICKERSON,E,temp,63,1961,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
1,NaN,BIGGS__EUGENE_L_25_01_02_2024,EUGENE,BIGGS,L,temp,25,1999,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
2,NaN,24TR60000100,NATASHA,TAYLOR,C,IL_COOK,47,1977,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
3,NaN,24TR60000091,LORENZO,GERARDO,A,IL_COOK,35,1989,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
4,NaN,24TR50000090,JENNIFER,HENKE,L,IL_COOK,42,1982,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False


In [176]:
df['middle_name'] = df['middle_name'].fillna('')
df['first_name'] = df['first_name'].fillna('')
df['last_name'] = df['last_name'].fillna('')
df

,id,case_id,first_name,last_name,middle_name,court_code,age,year_of_birth,email,phone,...,notes,violation,court,accidentCheckbox,commercialDriverLicence,ticket_img,user_id,report,details,cloudtalk_upload
0,NaN,DICKERSON__JOHN_E_63_01_02_2024,JOHN,DICKERSON,E,temp,63,1961,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
1,NaN,BIGGS__EUGENE_L_25_01_02_2024,EUGENE,BIGGS,L,temp,25,1999,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
2,NaN,24TR60000100,NATASHA,TAYLOR,C,IL_COOK,47,1977,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
3,NaN,24TR60000091,LORENZO,GERARDO,A,IL_COOK,35,1989,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
4,NaN,24TR50000090,JENNIFER,HENKE,L,IL_COOK,42,1982,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,NaN,230257248,NATHAN,GLODECK,STANLEY,CT21,18,2005,NaN,NaN,...,NaN,NaN,NaN,False,False,230257248.png,NaN,NaN,NaN,False
103,NaN,220460163,AMORION,OLIPHANT,JAVON,CT21,19,2004,NaN,NaN,...,NaN,NaN,NaN,False,False,220460163.png,NaN,NaN,NaN,False
104,NaN,180458951,BARBARA,WILSON,V,CT21,43,1980,NaN,NaN,...,NaN,NaN,NaN,False,False,180458951.png,NaN,NaN,NaN,False
105,NaN,GREER__LAMAR_L_28_01_01_2024,LAMAR,GREER,L,temp,28,1996,NaN,NaN,...,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,False


In [177]:
# df['phone_numbers_spokeo'] = df.apply(lambda x: preson_search(x['first_name'], x['last_name'], x['middle_name']), axis=1)
phone_numbers_spokeo = []
df = df.head(2)

In [178]:
for index, row in df.iterrows():
    try:
        result = await preson_search(page, row['first_name'], row['last_name'], row['middle_name'])
        print(f"user {row['first_name']} ==> ", result)
    except Exception as e:
        print(e)
        result = []

    phone_numbers_spokeo.append(result)

'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/Delaware/Delmar/p1414506781
'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/California/Santa-Ana/p2017110414536014371307633089762
'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/New-Jersey/Haskell/p10417352031
'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/Georgia/Cartersville/p3306422041
'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/Nebraska/Omaha/p2015020616042011892512431527417
'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/Indiana/Indianapolis/p339742401
'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Dickerson/New-Jersey/Howell/p373562201
'NoneType' object has no attribute 'query_selector' ==> https://www.spokeo.com/John-Di

In [179]:
phone_numbers_spokeo

[['(714) 894-6399',
  '(803) 356-1433 +4 phones',
  '(480) 839-4121 +3 phones',
  '(803) 371-9060 +4 phones',
  '(904) 744-7825 +4 phones',
  '(803) 822-8832 +6 phones',
  '(302) 258-8358 +1 phone',
  '(318) 934-3491 +3 phones',
  '(435) 729-9560 +5 phones',
  '(803) 920-0165 +8 phones',
  '(423) 383-8624 +7 phones',
  '(513) 594-7331 +5 phones',
  '(770) 808-4510',
  '(973) 627-6102',
  '(714) 260-1713 +8 phones',
  '(512) 385-1867 +1 phone',
  'See available information'],
 ['(910) 461-0094 +5 phones',
  '(402) 812-1311 +7 phones',
  'See available information',
  '(863) 293-4470',
  '(423) 741-3609 +1 phone',
  '(828) 712-4612 +8 phones',
  '(386) 986-3949 +1 phone',
  '(734) 850-0000 +1 phone',
  '(561) 601-8936 +1 phone',
  '(618) 549-3004 +2 phones',
  '(615) 390-8223 +3 phones',
  '(850) 585-3944 +1 phone',
  '(813) 505-0132 +4 phones',
  '(925) 980-8903 +3 phones',
  'See available information',
  '(316) 644-0921 +1 phone',
  '(734) 347-1999 +1 phone']]

In [132]:
df['phone_numbers_spokeo'] = phone_numbers_spokeo


/var/folders/c2/zqn3sbnd33j0_q_t7ss20yl40000gq/T/ipykernel_31128/3914125115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['phone_numbers_spokeo'] = phone_numbers_spokeo


In [133]:
df['phone_numbers_spokeo']


0    []
1    []
Name: phone_numbers_spokeo, dtype: object

In [134]:
df.to_csv('phone_numbers_spokeo.csv', index=False)

In [135]:
await browser.close()
await pw.stop()